# What is Handwritten Digit Recognition?
Handwritten digit recognition is the ability of computers to recognize human handwritten digits. It is a hard task for the machine because handwritten digits are not perfect and can vary from person to person. Handwritten digit recognition is the solution to this problem which uses the image of a digit and recognizes the digit present in the image.

# About the Dataset

This dataset is inspired by the MNIST database for handwritten digits. It consists of images representing digits from 0-9 produced using 2,990 google fonts files.

The dataset consists of a single file:

TMNIST_Data.csv

This file consists of 29,900 examples with labels and font names. Each row contains 786 elements: the first element represents the font name (ex-Chivo-Italic, Sen-Bold), the second element represents the label (a number from 0-9) and the remaining 784 elements represent the grayscale pixel values (from 0-255) for the 28x28 pixel image.



# What is Convolutional Neural Network

A Convolutional Neural Network (ConvNet/CNN) is a Deep Learning algorithm that can take in an input image, assign importance (learnable weights and biases) to various aspects/objects in the image, and be able to differentiate one from the other. The pre-processing required in a ConvNet is much lower as compared to other classification algorithms. While in primitive methods filters are hand-engineered, with enough training, ConvNets have the ability to learn these filters/characteristics.

A ConvNet's architecture is influenced by how the Visual Cortex is organised and is similar to the connectivity network of neurons in the human brain. While there are many complex CNN architectures like Resnet, Inception V3 and so on designed to achieve state-of-the-art performance on real world images, in this tutorial, we will create a custom Convolutional Neural Network Architecture using Pytorch.

We will be training and testing our model on the Typeface MNIST Dataset, which consists of over 29,000 samples of images of 0-9 digits in various fonts along with their labels.

#                          Architecture of Neural Network

<img src= "https://upload.wikimedia.org/wikipedia/commons/b/b6/Artificial_neural_network.png" alt ="Titanic" style='width: 700px;'>



This notebook is further divided into the following sections.

- Importing required Libraries
- Loading the dataset
- Creating a CNN Model
- Training our model
- Evaluating our model

# Importing All necessary librarirs

In [ ]:

#import pytorch
import torch

# torch functionalities used for building our CNN Model
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import torch.utils.data as data_utils

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# How to train your neural net faster?
We saw that the computationally intensive part of neural network is made up of multiple matrix multiplications. So how can we make it faster?

Since we will be building a CNN model, a GPU is required to speed up the training process. Before starting with the implementation, we verify the presence of an active GPU and set the device accordingly.

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:

#Reading data into pandas dataframe.
#data_path = "/kaggle/input/94-character-tmnist/94_character_TMNIST.csv"
data_path = "/kaggle/input/tmnist-typeface-mnist/TMNIST_Data.csv"
data = pd.read_csv(data_path)


# Performing EDA

In [ ]:
#A view of dataset.
data.head()

In [ ]:
#Determining Shape and Number of Sample in the Dataset
print(f"The Shape of the Dataframe is: {data.shape}") #Number of Samples, Number of Pixels(Features)
print(f"Number of Samples: {data.shape[0]}")

In [ ]:
# Number of unique character in the Dataframe
print(f"Number of unique character present in the Dataset: {len(data.labels.unique())}") #Number of Classes


In [ ]:
# Label distribution
data.labels.value_counts()

We observe that the labels are equally distributed with 2990 instances of each class. Hence, our dataset is balanced.

Now, let us reshape and modify our data so that it can be used as an input for our model. We normalise the data and reshape it into a 4 dimensional array such that it represents images stacked on to each other. We also reshape our labels into a 1 dimensional array.

In [ ]:
#Dropping redundant coloums like names and labels from the training frames and keeping only pixel values
X = data.drop(columns = {'names', 'labels'})/255
X = X.values.reshape(X.shape[0], 1, 28, 28)
y = data[['labels']].values.reshape((-1,))

Before proceeding, we split our data into training and testing sets using an 80:20 proportion.

In [ ]:
#split data into training and testing sets using the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

We now convert them to numpy arrays so that they can be further worked with.

In [ ]:
# Convert X_train, X_test, y_train, and y_test to PyTorch tensors
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

Before proceeding, we split our data into training and testing sets using an 80:20 proportion.

In [ ]:
#printing shape
print(X_train.shape, y_train.shape)

We verify our steps by checking the shape of our newly created torch tensors.

Now, let us visualise a sample image from our processed data.

In [ ]:

def visualize_image(data):
    # Display the image using matplotlib's imshow() function
    plt.imshow(data[0], cmap='gray_r')
    # Turn off the axis display in the plot
    plt.axis("off")
    # Show the plot with the displayed image
    plt.show()

# Call the visualize_image() function with the first image in X_train dataset
visualize_image(X_train[0])


#  Create a CNN Model
Create Data Loader
Dataloaders are utilized to feed our input data to our model for training, based on different parameters such as the batch size, if they should be suffled while being fed, and so on.

Here, we define our tensordatasets for the dataloader and then input our required parameters to prepare the data loaders.

In [ ]:
print(type(X_train))
print(type(y_train))


In [ ]:
# Create a training dataset from X_train and y_train
train = data_utils.TensorDataset(X_train, y_train)
# Create a testing dataset from X_test and y_test
test = data_utils.TensorDataset(X_test, y_test)


Once you have created your TensorDataset objects, you can pass them to data loaders in PyTorch to efficiently load and preprocess your data during model training or testing. Data loaders allow you to iterate over your dataset in batches, shuffle the data, and apply other data augmentation techniques as needed

In [ ]:

# Create a training data loader from the train dataset
# with a batch size of 128 and shuffling the data
train_loader = data_utils.DataLoader(train, batch_size=128, shuffle=True)

# Create a testing data loader from the test dataset
# with a batch size of 128 and shuffling the data
test_loader = data_utils.DataLoader(test, batch_size=128, shuffle=True)


To verify our data loaders, we print the input size of an iteration of training.

In [ ]:
#printing input size of an iteration of training
for images, labels in train_loader:
    print(images.shape, labels.shape)

data_iter = iter(train_loader)
images, labels = next(data_iter)
print(images.shape, labels.shape)



In [ ]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)     # First convolutional layer with input channels=1, output channels=32, and kernel size=3x3
        self.conv2 = nn.Conv2d(32, 64, 3)    # Second convolutional layer with input channels=32, output channels=64, and kernel size=3x3
        self.pool = nn.MaxPool2d(2, 2)       # Max pooling layer with kernel size=2x2 and stride=2
        self.fc1 = nn.Linear(64 * 12 * 12, 128)  # Fully connected layer with input size of 64x12x12 (output from previous layers) and output size of 128
        self.fc2 = nn.Linear(128, 10)        # Fully connected layer with input size of 128 and output size of 10 (for 10 classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))            # Apply ReLU activation to the output of the first convolutional layer
        x = self.pool(F.relu(self.conv2(x))) # Apply ReLU activation to the output of the second convolutional layer, then perform max pooling
        x = x.view(-1, 64 * 12 * 12)         # Flatten the tensor to a 1D vector
        x = F.relu(self.fc1(x))              # Apply ReLU activation to the output of the first fully connected layer
        x = self.fc2(x)                      # Output from the second fully connected layer (logits)
        return x

net = Net()                               # Create an instance of the neural network model
net.to(device)                           # Move the model to the specified device (e.g., GPU or CPU)


# Create Model

- Convolution layer (CONV)- The convolution layer (CONV) uses filters that perform convolution operations as it is scanning the input I with respect to its dimensions. Its hyperparameters include the filter size F and stride S. The resulting output O is called feature map or activation map.
- Pooling (POOL)- The pooling layer (POOL) is a downsampling operation, typically applied after a convolution layer, which does some spatial invariance. In particular, max and average pooling are special kinds of pooling where the maximum and average value is taken, respectively.

To create our CNN Model, we utilise the neural network module from pytorch and provide it with our own custom architecture as follows.

- Convolutional layer with 32 channels
- Convolutional layer with 64 channels
- MaxPooling layer to compress information
- Dense layer to flatten the information obtained
- Dense output layer with relu activation

We use Cross Entropy Loss as the loss function and Adam as the optimizer here. You can also try out other optimizers like SGD here 

In [ ]:

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)
optimizer = optim.Adam(net.parameters(), lr = 0.001)

Finally, inorder to deal with input data in the float format, we modify the format of our CNN model net to float.

In [ ]:
net = net.float()

# Train Model

Definitions:

- Epoch- In the context of training a model, epoch is a term used to refer to one iteration where the  model sees the whole training set to update its weights.


- Mini batch gradient descent- During the training phase, updating weights is usually not based on the  whole training set at once due to computation complexities or one data point due to noise issues. Instead, the update step is done on mini-batches, where the number of data points in a batch is a hyperparameter that we can tune.


- Loss function -In order to quantify how a given model performs, the loss function .L is usually used   to evaluate to what extent the actual outputs y are correctly predicted by the model outputs z.


- Cross-entropy lossIn the context of binary classification in neural networks, the cross-entropy loss 
 L(z,y) is commonly used and is defined as follows:


We now train the model for 15 epochs and print its loss and test accuracy for each 50 completed mini batches. I used 15 epochs as it provided a satisfactory learning curve for this dataset, but feel free to explore with other number as well.

In [ ]:
epoch_num = 0
actual_loss = 0.0
accuracy = 0.0

In [ ]:
#training model with 15 epochs

num_epochs = 15

# Create lists to store epoch number, loss, and accuracy for each epoch
epoch_log = []
loss_log = []
accuracy_log = []

for epoch in range(num_epochs):
    print(f'Starting Epoch: {epoch + 1}...')

    running_loss = 0.0   # Initialize running loss to 0 for each epoch

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)    # Move inputs to the specified device (e.g., GPU or CPU)
        labels = labels.to(device)    # Move labels to the specified device

        optimizer.zero_grad()        # Zero the gradients of the optimizer

        outputs = net(inputs.float())   # Forward pass to get predicted outputs from the model
        loss = criterion(outputs,labels)  # Compute the loss between predicted outputs and ground truth labels
        loss.backward()                 # Backward pass to compute gradients
        optimizer.step()                # Update model weights using the optimizer

        running_loss += loss.item()   # Add the current batch loss to the running loss for this epoch
        if i % 50 == 49:
            correct = 0
            total = 0
            
            with torch.no_grad():
                for data in test_loader:
                    images, labels = data
                    images = images.to(device)    # Move images to the specified device
                    labels = labels.to(device)    # Move labels to the specified device
                    outputs = net(images.float())  # Forward pass to get predicted outputs from the model
                    
                    _, predicted = torch.max(outputs.data, dim = 1)   # Get predicted labels with maximum probability
                    total += labels.size(0)   # Update total number of images
                    correct += (predicted == labels).sum().item()   # Update correct predictions count
                
                accuracy = 100 * correct / total   # Calculate accuracy for this epoch
                actual_loss = running_loss / 50   # Calculate average loss for this epoch
                epoch_num = epoch + 1   # Get current epoch number
                print(f'Epoch: {epoch_num}, Mini-Batches Completed: {(i + 1)}, Loss: {actual_loss:.3f}, Test Accuracy: {accuracy:.3f}%')
                
    epoch_log.append(epoch_num)   # Append epoch number to the epoch log
    loss_log.append(actual_loss)   # Append average loss to the loss log
    accuracy_log.append(accuracy)   # Append accuracy to the accuracy log


print('Training Completed.')

# Saving Model
Let's save our trained model to PATH so that it can be reused.

In [ ]:
PATH = './cnn_net.pth'
torch.save(net.state_dict(), PATH)

# Evaluating  Model
Reloading Model
To evaluate our trained CNN model, we reload our saved model.

In [ ]:
net = Net()
net.to(device)
net.load_state_dict(torch.load(PATH))

# How well our model makes prediction with test data?
Now, let us use our test dataset to determine the final accuracy of our trained model.

In [ ]:
correct = 0   # Initialize count of correct predictions to 0
total = 0     # Initialize count of total predictions to 0

with torch.no_grad():   # Disable gradient computation for efficiency
    for data in test_loader:   # Loop through the test dataset
        images, labels = data   # Get images and labels from the test dataset
        images = images.to(device)   # Move images to the specified device (e.g., GPU or CPU)
        labels = labels.to(device)   # Move labels to the specified device
        outputs = net(images.float())   # Forward pass to get predicted outputs from the model
        _, predicted = torch.max(outputs.data, 1)   # Get predicted labels with maximum probability
        label_size = labels.size(0)   # Get the number of labels in the current batch
        total += label_size   # Update total count of predictions
        correct += (predicted == labels).sum().item()   # Update count of correct predictions by summing up correct predictions in the current batch

accuracy = 100 * correct / total   # Calculate accuracy by dividing correct predictions by total predictions and multiplying by 100 to get percentage

print(f'Accuracy of the network on test images: {accuracy:.3}%')

As we observe, our model achieves an accuracy of 99.1% on the test images.

# Classification Observations
Let us also view some of the classifications our model performs on the test data.

In [ ]:
net.eval()   # Set the model to evaluation mode, which disables dropout and batch normalization

num_images = 5   # Number of images to visualize

with torch.no_grad():   # Disable gradient computation for efficiency
    for data in test_loader:   # Loop through the test dataset
        images, labels = data   # Get images and labels from the test dataset
        images = images.to(device)   # Move images to the specified device (e.g., GPU or CPU)
        labels = labels.to(device)   # Move labels to the specified device

        outputs = net(images.float())   # Forward pass to get predicted outputs from the model
        predictions = torch.argmax(outputs, dim=1)   # Get predicted labels with maximum probability

        for i in range(data[0].shape[0]):   # Loop through the images in the current batch
            pred = predictions[i].item()   # Get the predicted label for the current image
            label = labels[i]   # Get the actual label for the current image
            if(num_images > 0):   # Check if the number of images to visualize is greater than 0
                print(f'Actual Label: {pred}, Predicted Label: {label}')   # Print the actual and predicted label for the current image
                img = np.reshape(images[i].cpu().numpy(),[1,28,28])   # Convert the image tensor to a NumPy array and reshape it to [1, 28, 28] size
                visualize_image(img)   # Visualize the image using a custom function (e.g., visualize_image)
                num_images -= 1   # Decrement the number of images to visualize by 1


# Plotting training logs
To further understand and analyse the training of our model, let us plot the training logs for loss and accuracy.

In [ ]:
fig, ax1 = plt.subplots()   # Create a figure with a single subplot and return the figure object and axes object

plt.title("Accuracy & Loss vs Epoch")   # Set the title of the plot

ax2 = ax1.twinx()   # Create a twin y-axis on the right side of the plot

ax1.plot(epoch_log, loss_log, 'g-')   # Plot the epoch vs loss with green color
ax2.plot(epoch_log, accuracy_log, 'b-')   # Plot the epoch vs test accuracy with blue color on the twin y-axis

ax1.set_xlabel('Epochs')   # Set the x-axis label to 'Epochs'
ax1.set_ylabel('Loss', color='g')   # Set the left y-axis label to 'Loss' with green color
ax2.set_ylabel('Test Accuracy', color='b')   # Set the right y-axis label to 'Test Accuracy' with blue color

plt.show()   # Display the plot


Observations: As the Number of Epochs increases i.e number of times you run the dataset through model, test accuracy increases and loss decreases

# Plotting Confusion Matrix
Lastly, let us plot the confusion matrix to evaluate the classwise performance of our model and understand where it can do better.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

In [ ]:
pred_list = torch.zeros(0, dtype=torch.long, device='cpu')   # Create an empty tensor to store predicted labels with dtype long and device CPU
label_list = torch.zeros(0, dtype=torch.long, device='cpu')   # Create an empty tensor to store actual labels with dtype long and device CPU

with torch.no_grad():   # Turn off gradient computation during inference
    for i, (inputs, classes) in enumerate(test_loader):   # Iterate over the test data loader
        inputs = inputs.to(device)   # Move inputs to the specified device (e.g., CPU or GPU)
        classes = classes.to(device)   # Move actual labels to the specified device
        outputs = net(inputs.float())   # Pass inputs through the network to get predicted outputs
        _, preds = torch.max(outputs, 1)   # Get the predicted labels by finding the indices of maximum values along the second dimension (class probabilities)

        pred_list = torch.cat([pred_list, preds.view(-1).cpu()])   # Concatenate predicted labels to the pred_list tensor, reshaping to 1D and moving to CPU
        label_list = torch.cat([label_list, classes.view(-1).cpu()])   # Concatenate actual labels to the label_list tensor, reshaping to 1D and moving to CPU




Further, let us visualise this confusion matrix.

In [ ]:
conf_mat = confusion_matrix(label_list.numpy(), pred_list.numpy())   # Compute the confusion matrix using predicted and actual labels converted to NumPy arrays
class_names = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)   # Define the class names as a tuple of integers from 0 to 9
dataframe = pd.DataFrame(conf_mat, index=class_names, columns=class_names)   # Create a DataFrame from the confusion matrix, with class names as row and column indices


In [ ]:
#plotting confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")
 
plt.title("Confusion Matrix"), plt.tight_layout()
 
plt.ylabel("True Class"), 
plt.xlabel("Predicted Class")
plt.show()

## Observations:

He we infer that , 558 times the label was 0 and our model predicted it as 0.Similarly 567 times label was 1 and and model predicted 1. Highest value is 4 , which means four times the label was 8 (True class) and our model predicted it 3, which is considered as error. 

# Conclusion

- The accuracy on test test is 99.1% which is really good for CNN
- CNNs, or Convolutional Neural Networks, are specialized neural networks for processing data with a known grid-like topology, such as images or time-series data.
- CNNs consist of convolutional layers, pooling layers, and fully connected layers, and are commonly used in computer vision and other areas of deep learning.
- Convolutional layers perform convolution on the input data to extract local patterns or features.
- Pooling layers downsample the spatial dimensions of the data while preserving important features.
- Activation functions, such as ReLU, sigmoid, and tanh, introduce non-linearity into the network.
- We have successfully created a  CNN model using Pytorch.

# References
- https://towardsdatascience.com/convolutional-neural-networks-explained-9cc5188c4939
- https://www.youtube.com/watch?v=iqQgED9vV7k
- https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel